In [44]:
using StatsBase
using Combinatorics
using Plots

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/cut.jl")

# parameters

n = 100
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

kmax = 4
kmin = 1

fk = k->(2.*μ*k)^(-k)
fp = harmonicMean

Ω_dict = Dict()

for k = kmin:kmax, p in partitions(k)
    Ω_dict[p] = fk(sum(p))*fp(p)
end

Ω = buildΩ(Ω_dict; by_size=true)

Ω (generic function with 1 method)

In [45]:
# sample from the HSBM with these parameters, restricting to hyperedges of size no more than kmax
H = sampleSBM(Z, ϑ, Ω; kmax=kmax, kmin = kmin)
countEdges(H)

14997

In [3]:
# Create a random clustering with k clusters
include("jl/cut.jl")
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω)

-80682.35953347036

In [4]:
# PC: broken under the dictionary-defined Ω. Could add these manually if desired, but not sure whether this is a problem or not. 

# Quick checks on the group evaluation function
# p4 = [4,0,0,0]
# p3 = [3,1,0,0]
# p2 = [2,2,0,0]
# p1 = [2,1,1,0]
# p0 = [1,1,1,1]
# @show Ω(p4;mode="partition"), Ω(p3;mode="partition"),Ω(p2;mode="partition"),Ω(p1;mode="partition"),Ω(p0;mode="partition")
# I'm going to think about some group interactions functions where the output is ordered
# the same as the way I've ordered here, i.e. Ω(p(i+1) >= Ω(p(i)
# Ordering by majorization seems natural for clustering applications

# PC: the harmonic mean will give you zeros whenever there is a group without any nodes in it. I'd suggest writing this informal experiment as in the next block: 

In [5]:
# still works because we haven't added zero-padding. 

for i = 1:4, p in partitions(4, i)
    print(p, " ")
    println(Ω(p;mode="partition"))
end

# an interesting note from this is that the harmonic mean doesn't actually respect the majorization order....

[4] 1.1800991682260451e-5
[3, 1] 1.0982060201735035e-5
[2, 2] 1.1800991682260451e-5
[2, 1, 1] 9.923411602051714e-6
[1, 1, 1, 1] 8.344561243252407e-6


In [49]:
@time T1 = first_term_eval(H,c,Ω)

  0.141174 seconds (374.78 k allocations: 26.201 MiB)


-146854.85601820663

In [48]:
# Here's a faster way to calculate it...for now requires a different way to store the hypergraph and Omega

@time Hyp, w = hyperedge_formatting(H)

@time T2 = first_term_v2(Hyp,w,c,Ω)

T1 ≈ T2

  0.005921 seconds (29.00 k allocations: 1.211 MiB)
  0.074355 seconds (299.94 k allocations: 23.954 MiB, 27.59% gc time)


false